### 1. 결정 트리(Decision Tree)

* 본 텀프로젝트의 목적은 **회귀 결정 트리**를 구현하는 것

* 사이킷런 [tree](https://scikit-learn.org/stable/modules/tree.html)을 참조하면 결정 나무 모델에 대한 설명을 아래 사진과 같이 확인 가능

<center>
<img src="https://drive.google.com/uc?id=17gQ6QQyMrrt2pNIRhy8kCLdQuICOxGrP" width="900">
</center>

* 결정 트리(Decision Tree)는 스무고개 게임과 유사하여 룰 기반의 프로그램에 적용되는 `if`, `else`를 자동으로 찾아내(분할 규칙) 예측을 위한 알고리즘

* 결국 결정트리를 생성하는 것은 주어진 특성공간을 분할 규칙에 따라 분할하는 것과 같음

* 학습 데이터 $D=\{(x_i,y_i)|1\le i \le m\}$의 특성벡터 $x_i\, (1\le i \le m)$를 포함하는 특성공간  $\mathcal X$를 어떤 <span style="color:blue"> 분할 규칙(splitting rule)</span>에 따라 겹치지 않는 작은 영역 $\mathcal R_i$로 나눔
$$\mathcal X = \mathcal R_1 \cup \mathcal R_2 \cup \cdots \cup \mathcal R_N$$


* 회귀문제인지 분류문제인지에 따라, 임의의 샘플벡터 $x$에 대해 다음과 같이 예측 
> * 회귀의 경우: 샘플 $x$가 속하는 작은 영역 $\mathcal R_i$에 대해, **이 영역에 속하는 훈련샘플 $x_j$의 $y_j$값의 평균**으로 예측 
$$\hat y = \dfrac 1 {r_i} \sum_{x_j \in \mathcal R_i}y_j, \quad (r_i=\bigl|\{(x_j,y_j)\in D|x_j\in \mathcal R_i\}\bigr|)$$
$$ $$
> * 분류의 경우: 샘플 $x$가 속하는 작은 영역 $\mathcal R_i$에 대해, $\mathcal R_i$에 속하는 훈련샘플에 대한 레이블 중 가장 많이 나타나는 레이블  
$$ $$


In [158]:
from sklearn.datasets import make_friedman1
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeRegressor

import numpy as np 

#### 1-1) 트리구조를 구현하기 위해 트리노드 클래스를 구현  

* 클래스의 속성와 메소드 
> * 결정트리에서의 각 노드는 특성공간 $\mathcal X$의 부분집합에 대응하므로 이 부분집합에 속하는 훈련 데이터셋 `self.X`와 레이블 `self.y`가 필요  
> * 결정트리에서 노드가 위치하는 깊이: `self.depth`  
> * 노드에서 분할규칙 $s_{j}^{x_{ij}}$를 결정하는 속성 `self.j`와 기준값 $x_{ij}$를 나타내는 `self.xi`  
> * 자식노드: `self.left`, `self.right`  
> * 노드에 대응되는 예측함수: `self.predictor`
> * 노드에 대응하는 집합 $\mathcal A$에서의 예측함수 $g^{\mathcal A}$를 이용하여 $\mathcal A$에 속하는 훈련샘플에 대한 **SSE(Sum of Squares for Error)**를 계산하는 함수 `CalculateLoss`  


In [159]:
class TNode:
    def __init__(self,depth, X, y):

        self.depth = depth                      # 트리 max depth
        self.X = X                              # train_X (Feature)
        self.y = y                              # train_y (Label)
        self.xi = None                          # 분할 인덱스
        self.left = None                        # 왼쪽 자식 노드
        self.right = None                       # 오른쪽 자식 노드
        self.predictor = None                   # 예측 함수

    def CalculateLoss(self):
        if len(self.y)==0:
            return 0
        else:
            ####### Empty Module.1 #######
            # 주어진 타겟 변수(self.y)와 평균 값(np.mean(self.y))의 차이를 계산하고 차이를 제곱(np.power())하여 모든 차이의 합계(np.sum()) = SSE를 반환 
            return np.sum(np.power(self.y - np.mean(self.y), 2))
            ##############################

### 1-2. 결정트리(Decision Tree) 생성 함수 

* 주어진 노드에 대응되는 영역 $\mathcal A$를 분할하여 훈련 데이터셋에 대한 손실값이 최소가 되는 분할규칙 $s_j^{x_{ij}}$를 찾아서 `j`,`xi`를 반환해주는 함수 `CalculateOptimalSplit`를 구현 


* `CalculateOptimalSplit`함수를 이용하여 구한 분할규칙에 따라 $\mathcal A$의 분할 $\mathcal A_T$, $\mathcal A_F$에 대응되는 노드를 생성하기 위해, 현재 노드에 대응되는 영역 $\mathcal A$에 속하는 훈련 데이터셋 `X`,`y`를 영역 $\mathcal A_T$에 속하는 훈련 데이터셋 `Xt`,`yt`와 영역 $\mathcal A_F$에 속하는 훈련 데이터셋 `Xf`,`yf`로 나누어주는 함수 `DataSplit`를  구현 

In [160]:
def DataSplit(X, y, j, xi):
    ####### Empty Module.2 #######
    # 주어진 데이터셋 X와 타겟 변수 y를 특정
    # 특성 j의 값이 분할 임계값 xi 이하인 데이터 포인트의 인덱스를 선택하고, 해당 인덱스를 사용하여 데이터셋을 분할
    # 분할 결과로서 분할된 데이터셋 Xt, Xf, yt, yf를 반환
    ids = X[:, j] <= xi
    Xt = X[ids]
    Xf = X[~ids]
    yt = y[ids]
    yf = y[~ids]
    ##############################
    return Xt, yt, Xf, yf

def CalculateOptimalSplit(node):
    # 주어진 노드 node에서 최적 분할 기준을 계산
    # 모든 특성과 가능한 모든 분할 임계값을 반복하면서 최적의 분할 기준 찾기
    X = node.X
    y = node.y
    best_feature = 0
    bext_xi = X[0, best_feature]
    best_split_val = node.CalculateLoss()
    
    m,n = X.shape
    
    for j in range(0,n):
        for i in range(0,m):
            xi = X[i,j]
            Xt, yt, Xf, yf = DataSplit(X,y,j,xi)

            #  분할된 데이터셋에 대해 새로운 임시 노드 tmpt와 tmpf를 생성하고, 각각의 손실 함수 값을 계산
            # loss_t는 tmpt의 손실 함수 값이고, loss_f는 tmpf의 손실 함수 값
            tmpt = TNode(0, Xt, yt)
            tmpf = TNode(0, Xf, yf)
            loss_t = tmpt.CalculateLoss()
            loss_f = tmpf.CalculateLoss()
            curr_val = loss_t + loss_f # 현재의 손실함수 값
            ####### Empty Module.3 #######
            # 현재의 손실 함수 값과 비교하여 더 작은 손실 함수 값이 나오는 경우, 최적의 분할 기준을 업데이트
            # curr_val = loss_t + loss_f로 계산된 현재의 손실 함수 값이 최적 분할 기준 best_split_val보다 작은 경우에는 best_split_val, best_feature, best_xi를 갱신
            if curr_val < best_split_val:
                best_split_val = curr_val # loss 업데이트
                best_feature = j # best_feature 업데이트
                best_xi = xi # best_xi 업데이트
            ##############################

    return best_feature, best_xi

### 1-3. 결정트리(Decision Tree) 생성 함수 

* 현재 노드의 자식노드를 설정하고, `Construct_Subtree`를 재귀적으로 사용함으로써 트리구조를 구현 
> * 미리 설정된 `depth`에 도달하거나, 더 이상 분할할 수 없을 때(즉, `self.X` 또는 `self.y`의 개수가 $1$) 리프노드에 대한 예측함수를 설정.

In [161]:
# 현재 노드의 자식 노드를 설정하고, 재귀적으로 Construct_Subtree() 함수를 호출하여 트리 구조를 구축
def Construct_Subtree(node, max_depth):
    if (node.depth == max_depth or len(node.y) == 1): # node의 깊이가 max_depth에 도달했거나 리프 노드일 때
        ####### Empty Module.4 #######
        # 노드의 깊이(node.depth)가 max_depth에 도달하거나 노드의 타겟값 y의 길이가 1인 경우를 체크
        # 위의 조건이 True면, 해당 노드는 리프노드이며 노드 내부에 있는 타겟값들 node.y의 평균을 계산하여 node.predictor에 저장
        # 이렇게 저장된 node.predictor는 해당 노드에서의 예측값으로 사용
        node.predictor = np.mean(node.y) # node 내부에 있는 y값들의 평균을 활용하여 예측 수행
        ##############################
    else:
        # 만약 위의 조건이 False면(리프 노드가 아닌 경우), CalculateOptimalSplit(node) 함수를 호출하여 현재 노드에서의 최적 분할 기준을 찾습니다.
        # 반환되는 j와 xi는 최적 분할 기준으로, node.j와 node.xi에 저장
        j, xi = CalculateOptimalSplit(node)                
        node.j = j
        node.xi = xi
        # DataSplit(node.X, node.y, j, xi) 함수를 호출하여 최적 분할 기준을 기준으로 데이터셋을 분할
        Xt, yt, Xf, yf = DataSplit(node.X, node.y, j, xi)  

        if (len(yt)>0): 
            # Xt, yt에 샘플이 존재하는지 확인. 즉, 왼쪽 자식 노드를 생성해야하는 경우
            ####### Empty Module.5 #######
            # node.left에 새로운 TNode 객체를 생성. 이때 노드의 깊이를 현재 노드의 깊이(node.depth)에 1을 더한 값으로 설정
            # node.left에 분할된 데이터셋 Xt, yt를 전달하여 초기화
            node.left = TNode(node.depth + 1, Xt, yt)

            # Construct_Subtree(node.left, max_depth)를 재귀적으로 호출하여 왼쪽 자식 노드의 서브트리를 구성
            Construct_Subtree(node.left, max_depth)
            ##############################
    
        if (len(yf)>0): 
            # Xf, yf에 샘플이 존재하는지 확인. 즉, 오른쪽 자식 노드를 생성해야하는 경우
            ####### Empty Module.6 #######
            # node.right에 새로운 TNode 객체를 생성. 이때 노드의 깊이를 현재 노드의 깊이(node.depth)에 1을 더한 값으로 설정
            # node.right에 분할된 데이터셋 Xf, yf를 전달하여 초기화
            node.right = TNode(node.depth + 1, Xf, yf)

            # Construct_Subtree(node.right, max_depth)를 재귀적으로 호출하여 오른쪽 자식 노드의 서브트리를 구성
            Construct_Subtree(node.right, max_depth)
            ##############################

    return node

### 1-4.결정트리(Decision Tree) 예측기 구현  

* 학습된 결정트리를 이용하여 예측기 `Predict`를 재귀적으로 구현 

In [162]:
def Predict(X, node):
    ####### Empty Module.7 #######
    # 현재 노드가 왼쪽 자식만 가지고 있고 오른쪽 자식이 없는 경우
    # 왼쪽 자식 노드로 이동해야하는 경우
    if (node.right == None and node.left != None):
        return Predict(X, node.left)                                       # 재귀적으로 왼쪽 node에 대해서 Predict 함수 호출
    
    # 현재 노드가 오른쪽 자식만 가지고 있고 왼쪽 자식이 없는 경우
    # 오른쪽 자식 노드로 이동해야하는 경우
    if (node.right != None and node.left == None):
        return Predict(X, node.right)                                         # 재귀적으로 오른쪽 node에 대해서 Predict 함수 호출
    
    # 현재 노드가 정점인 경우, 즉, 현재 노드가 예측값을 가지고 있는 경우
    if (node.right == None and node.left == None):
        return node.predictor 
    # 위의 조건문들을 모두 만족하지 않는 경우는 현재 노드가 내부 노드인 경우    
    else:
        # 만약 X[node.j]가 node.xi보다 작거나 같은 경우, 왼쪽 자식 노드로 이동하여 재귀적으로 Predict 함수를 호출
        if (X[node.j] <= node.xi):
            return Predict(X, node.left) # 재귀적으로 왼쪽 node에 대해서 Predict 함수 호출
        # 그렇지 않으면 오른쪽 자식 노드로 이동하여 재귀적으로 Predict 함수를 호출
        else:
            return Predict(X, node.right) # 재귀적으로 오른쪽 node에 대해서 Predict 함수 호출
    ##############################

### 1-5. 실습 데이터셋 생성 및 결정트리 실습 

* `sklearn.datasets.make_friedman1` 함수를 이용하여 특성 개수가 $5$인 샘플 $1000$개를 생성하고, 이 중 $500$개를 훈련 데이터셋, 나머지를 테스트 데이터셋으로 사용 

In [163]:
def makedata():
    n_samples = 1000
    X, y = make_friedman1(n_samples = n_samples, n_features = 5, noise=1.0, random_state=100)
    return train_test_split(X, y, test_size=0.5, random_state=3)

### 1-6. `sklearn.tree` 모듈의 `DecisionTreeRegressor`와 비교 

In [164]:
X_train, X_test, y_train, y_test = makedata()

# 결정트리 깊이 설정
max_depth = 10

# depth 0에서 루트노드 생성 
treeRoot = TNode(0, X_train, y_train)

# 결정트리 학습 
Construct_Subtree(treeRoot, max_depth)

# 예측 
y_hat = np.zeros(len(X_test))

for i in range(len(X_test)):
    y_hat[i] = Predict(X_test[i], treeRoot)

regTree = DecisionTreeRegressor(max_depth = 10, random_state=0)

regTree.fit(X_train, y_train)
y_hat2 = regTree.predict(X_test)

MSE_scratch = np.mean(np.power(y_hat-y_test,2))
MSE_scikit = np.mean(np.power(y_hat2-y_test, 2))

print("사이킷런 결정트리:loss= {:.3f}".format(MSE_scikit))
print("직접구현 결정트리:loss= {:.3f}".format(MSE_scratch))

사이킷런 결정트리:loss= 7.632
직접구현 결정트리:loss= 7.731


### 2. 앙상블-배깅(Bagging)

* 예측기의 성능을 향상시키기 위해 여러개의 훈련데이터셋 각각을 이용하여 학습시킨 예측기를 결합하여 예측하는 방법을 배깅(Bagging)방식

* 배깅방식은 특히 가지치기가 이루어지지 않은 결정트리와 같이 훈련데이터셋의 훈련샘플의 작은 변화에 민감하게 영향을 받는 예측기들을 개선할 때 유용한 방식 

> * 회귀문제에 대한 결정트리 예측기의 경우를 예로 들어 배깅방식을 설명하면 다음과 같다. 
>> * 훈련데이터셋 $\mathcal D$가 어떤 분포를 따르는 랜덤집합이라하고, $\mathcal D_1,\cdots,\mathcal D_n$가 동일한 분포에 iid라 하자. 각 $\mathcal D_i\, (1\le i \le n)$를 훈련데이터셋으로 학습시킨 예측기를 $g_{\mathcal D_i}\, (1\le i \le n)$이라 할 때, 
$$g_{\rm{avg}}(\mathbf x) = \dfrac 1 n \sum_{i=1}^n g_{\mathcal D_i}(\mathbf x)$$
와 같이 예측기 $g_{\rm{avg}}$를 정의하면 큰 수의 법칙에 따라 예측기 $g_{\rm avg}$는 $n$이 커질 때 $g^*:=\rm{E}(g_{\mathcal D})$로 수렴하게 된다. 이때, 아래 정리에서 알 수 있듯이 서로 다른 훈련데이터셋으로 학습시킨 예측기의 평균값으로 예측을 하면 보다 나은 예측기를 구성할 수 있다.    
$$ $$

> * 이 때 현실적으로 동일한 분포 iid를 따르는 여러 개의 훈련데이터셋 $\mathcal D_1,\cdots,\mathcal D_n$ 을 얻는 것이 어려움.   
이를 극복하기 위해 $m$개의 훈련샘플로 이루어진 하나의 훈련데이터셋이 주어질 때, 이 훈련데이터셋에서 $m$개의 훈련샘플을 복원추출로 뽑는 과정을 $n$번 반복하여 랜덤 훈련데이터셋 $\mathcal D_1^*,\cdots, \mathcal D_n^*$을 구성하고    
$$ g_{bag}(\mathbf x)= \dfrac 1 n \sum_{i=1}^n g_{\mathcal D_i^*}(\mathbf x)$$
와 같이 예측기를 구성하는 것을 부트스트랩을 통해 종합한(bootstrapped aggregated) 예측기 또는 bagged 예측기라고 부른다.   



In [165]:
# bag 예측기 구성 (500개의 예측기 구성)

n_estimators = 500 
bag = np.empty((n_estimators), dtype=object)

# 루프를 통해 각 예측기를 구성
for i in range(n_estimators):

    ####### Empty Module.8 #######
    # 복원 추출을 이용하여 랜덤한 훈련 데이터셋 정의 (bootstrap)
    # replace=True를 지정하여 복원 추출을 수행
    ids = np.random.choice(len(X_train), size=len(X_train), replace=True) # np.random.choice 이용하여 인덱스 먼저 정의, 복원 추출로 진행, size는 len(X_train)만큼 샘플링

    # 인덱스(ids)를 이용하여 훈련 데이터셋에서 실제 데이터를 샘플링하여 X_boot과 y_boot을 생성
    X_boot = X_train[ids] # 인덱스 이용하여 데이터 샘플링
    y_boot = y_train[ids] # 인덱스 이용하여 데이터 샘플링
    ##############################

    bag[i] = DecisionTreeRegressor()
    bag[i].fit(X_boot, y_boot)
    
####### Empty Module.9 #######

# yhatbag 배열을 생성. 이 배열은 초기값으로 모두 0으로 설정
yhatbag = np.zeros(len(X_test)) # 500개의 예측기를 이용하여 yhatbag 생성 반복문 이용하거나 np의 mean 함수 사용할 것  

# for 루프를 통해 각 예측기를 사용
for i in range(n_estimators):
    # X_test에 대한 예측값 구하기
    # predict 메서드를 사용하여 각 예측기의 예측값을 계산하고, 이를 yhatbag에 누적
    yhatbag += bag[i].predict(X_test)

# yhatbag를 예측기의 개수 n_estimators로 나누어 평균값을 계산. 이는 배깅(Bagging)을 통해 얻은 예측기들의 평균 예측값을 나타냄
yhatbag = yhatbag / n_estimators

##############################

MSE_bagging = np.mean(np.power(yhatbag-y_test, 2))

In [166]:
print("사이킷런 결정트리:loss= {:.3f}".format(MSE_scikit))
print("직접구현 결정트리:loss= {:.3f}".format(MSE_scratch))
print("직접구현-배깅방식:loss= {:.3f}".format(MSE_bagging))

사이킷런 결정트리:loss= 7.632
직접구현 결정트리:loss= 7.731
직접구현-배깅방식:loss= 3.240
